# Voice Controlled Robotic Arm for Raspberry Pi Pico
This notebook controls a robotic arm via voice commands using Raspberry Pi Pico

## Import Required Libraries for Raspberry Pi Pico

In [ ]:
import speech_recognition as sr
import serial
from pynput import keyboard
import time

# Libraries for Raspberry Pi Pico serial communication
# The machine library is used for MicroPython on Pico
# For Python on the host computer controlling Pico, we use pyserial

## Configure Serial Communication with Raspberry Pi Pico

In [ ]:
# Configuration for Raspberry Pi Pico
# Pico communicates via USB Serial at 115200 baud (standard for Pico)
PICO_PORT = 'COM15'  # Change this to your Pico's COM port
PICO_BAUD_RATE = 115200  # Standard baud rate for Pico via USB

# Connection handler with error management
pico_connection = None

def connect_to_pico():
    """Establish serial connection to Raspberry Pi Pico"""
    global pico_connection
    try:
        pico_connection = serial.Serial(PICO_PORT, PICO_BAUD_RATE, timeout=1)
        time.sleep(2)  # Wait for Pico to reset
        print(f"Connected to Raspberry Pi Pico on {PICO_PORT}")
        return True
    except Exception as e:
        print(f"Failed to connect to Pico: {e}")
        print(f"Available ports - Make sure Pico is connected via USB")
        return False

def disconnect_from_pico():
    """Close serial connection to Pico"""
    global pico_connection
    if pico_connection:
        pico_connection.close()
        print("Disconnected from Pico")

## Voice Recognition and Coordinate Processing

In [ ]:
def create_correction_dictionary():
    """Create dictionary for correcting common speech recognition errors in Spanish"""
    return {
        "UNO": "1", "DOS": "2", "TRES": "3", "CUATRO": "4",
        "CINCO": "5", "SEIS": "6", "SIETE": "7", "OCHO": "8",
        "BE": "B", "VE": "B", "CE": "C", "DE": "D", "DE ": "D",
        "ES": "E", "ES ": "E", "EFE": "F", "GE": "G", "ACHE": "H", "HACHE": "H",
        "X": "*", "POR": "*",
        "A 6": "A6", "A SEIS": "A6",
    }

def recognize_coordinates():
    """Recognize chess coordinates from voice input using Google Speech Recognition"""
    corrections = create_correction_dictionary()
    recognizer = sr.Recognizer()
    
    print("Press 'H' to start speaking...")
    
    # Wait for user to press 'H'
    with keyboard.Listener(on_press=handle_keyboard_input) as listener:
        listener.join()
    
    if not continue_listening:
        return []
    
    print("Ready to listen...")
    
    try:
        with sr.Microphone() as source:
            audio = recognizer.listen(source)
        
        # Use Google Speech Recognition with Spanish language
        text = recognizer.recognize_google(audio, language="es-ES")
        text = text.upper()
        print(f"You said: {text}")
        
        # Split text into words
        words = text.split()
        
        # Apply corrections
        corrected_words = [corrections[word] if word in corrections else word for word in words]
        
        return corrected_words
    
    except sr.UnknownValueError:
        print("Could not understand audio")
        return []
    except sr.RequestError as e:
        print(f"Speech recognition error: {e}")
        return []

# Global variable for keyboard control
continue_listening = True

def handle_keyboard_input(key):
    """Handle keyboard input for voice control"""
    global continue_listening
    try:
        if hasattr(key, 'char'):
            if key.char == 'h' or key.char == 'H':
                continue_listening = True
                return False  # Stop listener
            elif key.char == 'q' or key.char == 'Q':
                continue_listening = False
                return False  # Stop listener
    except AttributeError:
        pass

## Coordinate Validation and Transformation

In [ ]:
def validate_and_process_coordinates(words):
    """Validate chess coordinates and process them"""
    coordinates = []
    is_capture = False
    
    # Check for capture move indicator
    words_copy = words.copy()
    if '*' in words_copy:
        is_capture = True
        words_copy.remove('*')
    
    # Validate coordinate count
    if not is_capture and len(words_copy) != 2:
        print("You need to provide exactly two valid coordinates")
        return None
    
    # Validate each coordinate
    for word in words_copy:
        if len(word) == 2 and word[0] in "ABCDEFGH" and word[1] in "12345678":
            coordinates.append(word)
            print(f"Coordinate '{word}' is valid")
        else:
            print(f"Coordinate '{word}' is invalid")
    
    # Verify we have the right number of coordinates
    if is_capture and len(coordinates) >= 1:
        coordinates.append("1")  # Capture flag
        return coordinates
    elif not is_capture and len(coordinates) == 2:
        coordinates.append("0")  # Standard move flag
        return coordinates
    else:
        print("Error: Could not process coordinates properly")
        return None

def transform_for_pico(coordinates):
    """Transform coordinates to Pico-compatible format"""
    if not coordinates or len(coordinates) < 3:
        return None
    
    # Replace special characters if needed
    start_coord = coordinates[0].replace('J', 'D').replace('K', 'A')
    end_coord = coordinates[1].replace('J', 'D').replace('K', 'A')
    move_type = coordinates[2]
    
    return [start_coord, end_coord, move_type]

## Send Commands to Raspberry Pi Pico

In [ ]:
def send_command_to_pico(coordinates):
    """Send validated coordinates to Raspberry Pi Pico"""
    global pico_connection
    
    if not pico_connection or not pico_connection.is_open:
        print("Error: Not connected to Pico")
        return False
    
    if len(coordinates) != 3:
        print("Error: Invalid coordinate format")
        return False
    
    start_coord = coordinates[0]
    end_coord = coordinates[1]
    move_type = coordinates[2]
    
    # Format message for Pico
    message = f"{start_coord} {end_coord} {move_type}\n"
    
    try:
        pico_connection.write(message.encode())
        print(f"Sent to Pico: {message.strip()}")
        
        # Wait for acknowledgment from Pico
        response = pico_connection.readline().decode().strip()
        if response:
            print(f"Pico response: {response}")
        
        return True
    except Exception as e:
        print(f"Error sending to Pico: {e}")
        return False

## Main Control Loop

In [ ]:
def main_voice_control_loop():
    """Main loop for voice-controlled robotic arm"""
    global continue_listening
    
    # Connect to Pico
    if not connect_to_pico():
        print("Cannot proceed without Pico connection")
        return
    
    continue_listening = True
    
    try:
        while continue_listening:
            print("\n" + "="*50)
            print("Press 'H' to record coordinates, 'Q' to quit")
            print("="*50)
            
            # Get voice input
            words = recognize_coordinates()
            
            if not words:
                print("No input received")
                continue
            
            # Process and validate coordinates
            coordinates = validate_and_process_coordinates(words)
            
            if coordinates:
                # Transform for Pico
                pico_coords = transform_for_pico(coordinates)
                
                if pico_coords:
                    # Send to Pico
                    send_command_to_pico(pico_coords)
                    time.sleep(1)  # Wait for execution
            
            if not continue_listening:
                break
    
    except KeyboardInterrupt:
        print("\nProgram interrupted")
    
    finally:
        disconnect_from_pico()
        print("Program ended")

# Run main loop
main_voice_control_loop()